In [1]:
import numpy as np
import pandas as pd
from tqdm.autonotebook import tqdm

tqdm.pandas()

/Users/far/.pyenv/versions/miniconda3-4.3.30/envs/spec-top-econ-data-vis/lib/python3.7/site-packages/ipykernel_launcher.py:3: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/far/.pyenv/versions/miniconda3-4.3.30/envs/spec-top-econ-data-vis/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
HRF = pd.read_csv('Hotel_Reviews.csv')
HRF.head()

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,21,9,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,52.360576,4.915968
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,Great location in nice surroundings the bar a...,26,1,3.8,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3 days,52.360576,4.915968
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/24/2017,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,140,1403,Amazing location and building Romantic setting,8,3,6.7,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",10 days,52.360576,4.915968


In [3]:
[' '.join(address.split()[-5:])
 for address in list(HRF['Hotel_Address'].unique()) if 'United Kingdom' in address]

['London SW5 9NB United Kingdom',
 'London EC4Y 1LL United Kingdom',
 'London W2 3BA United Kingdom',
 'London WC1B 5BE United Kingdom',
 'London SE1 7RY United Kingdom',
 'London WC2B 4BZ United Kingdom',
 'London W1J 7QY United Kingdom',
 'London W2 3JP United Kingdom',
 'London W8 5DL United Kingdom',
 'London WC1X 9HX United Kingdom',
 'London W6 8DR United Kingdom',
 'London SW1Y 4HX United Kingdom',
 'London SE10 0TW United Kingdom',
 'London SW3 1PT United Kingdom',
 'London W1T 3NP United Kingdom',
 'London SW5 0LA United Kingdom',
 'London W1D 3BR United Kingdom',
 'London EC4V 5AJ United Kingdom',
 'London SW7 3ER United Kingdom',
 'London WC2H 9HB United Kingdom',
 'London SW1E 5BW United Kingdom',
 'London EC3N 2NR United Kingdom',
 'London W1D 6QF United Kingdom',
 'London SW14 7RT United Kingdom',
 'London NW1 2AJ United Kingdom',
 'London SW7 4ER United Kingdom',
 'London SW7 5AG United Kingdom',
 'London E1 6JQ United Kingdom',
 'London SE11 5AL United Kingdom',
 'Londo

In [4]:
HRF['Reviewer_Nationality'].unique()

array([' Russia ', ' Ireland ', ' Australia ', ' United Kingdom ',
       ' New Zealand ', ' Poland ', ' Belgium ', ' Norway ', ' France ',
       ' Italy ', ' Canada ', ' Netherlands ', ' Germany ',
       ' United States of America ', ' Hungary ', ' Israel ',
       ' Switzerland ', ' India ', ' United Arab Emirates ', ' Malaysia ',
       ' Denmark ', ' South Africa ', ' Rwanda ', ' Finland ',
       ' Portugal ', ' Luxembourg ', ' Spain ', ' Malta ', ' China ',
       ' Turkey ', ' Ukraine ', ' Gibraltar ', ' Argentina ',
       ' Abkhazia Georgia ', ' Isle of Man ', ' Sweden ', ' Croatia ',
       ' Saudi Arabia ', ' Lithuania ', ' Slovenia ', ' Panama ',
       ' Austria ', ' Brazil ', ' Taiwan ', ' Iceland ', ' Iran ',
       ' Liechtenstein ', ' Chile ', ' Hong Kong ',
       ' United States Minor Outlying Islands ', ' Morocco ', ' Oman ',
       ' Lebanon ', ' Qatar ', ' Nigeria ', ' Romania ',
       ' Czech Republic ', ' Greece ', ' Thailand ', ' Singapore ',
       ' Guerns

In [5]:
def get_boundingbox_country(country, output_as='boundingbox'):
    """
    get the bounding box of a country in EPSG4326 given a country name

    Parameters
    ----------
    country : str
        name of the country in english and lowercase
    output_as : 'str
        chose from 'boundingbox' or 'center'. 
         - 'boundingbox' for [latmin, latmax, lonmin, lonmax]
         - 'center' for [latcenter, loncenter]

    Returns
    -------
    output : list
        list with coordinates as str
    """
    # create url
    url = '{0}{1}{2}'.format('http://nominatim.openstreetmap.org/search?country=',
                             country,
                             '&format=json&polygon=0')
    # response of an array
    response = requests.get(url).json()
    if len(response) == 0:
        return None
    # use the first one
    response = response[0]

    # parse response to list
    if output_as == 'boundingbox':
        lst = response[output_as]
        output = [float(i) for i in lst]
    if output_as == 'center':
        lst = [response.get(key) for key in ['lat', 'lon']]
        output = [float(i) for i in lst]
    return output

# Usage
# In [42]: get_boundingbox_country(country='netherlands')
# Out[42]: [11.777, 53.7253321, -70.2695876, 7.2274985]

# In [43]: get_boundingbox_country(country='holland')
# Out[43]: [11.777, 53.7253321, -70.2695876, 7.2274985]

# In [44]: get_boundingbox_country(country='nl')
# Out[44]: [11.777, 53.7253321, -70.2695876, 7.2274985]

# In [45]: get_boundingbox_country(country='nl', output_as='center')
# Out[45]: [52.5001698, 5.7480821]

In [ ]:
reviewer_nationalities = HRF['Reviewer_Nationality'].unique()

country_lat_lon_map = dict()

for v in tqdm(reviewer_nationalities):
    country = v.lower().strip()
    lat_lon = get_boundingbox_country(country=country, output_as='center')
    if lat_lon is not None:
        lat, lon = lat_lon
        country_lat_lon_map[country] = [lat, lon]

country_lat_lon_map

In [ ]:
import pickle

# Save country lat lon map
with open('country_lat_lon.pickle', 'wb') as pickle_file:
    pickle_file.write(pickle.dumps(country_lat_lon_map))

In [6]:
import pickle

# Load country lat lon map
with open('country_lat_lon.pickle', 'rb') as pickle_file:
    country_lat_lon_map = pickle.load(pickle_file)

country_lat_lon_map

{'russia': [64.6863136, 97.7453061],
 'ireland': [52.865196, -7.9794599],
 'australia': [-24.7761086, 134.755],
 'united kingdom': [55.367000000000004, -3.96141844454237],
 'new zealand': [-41.5000831, 172.8344077],
 'poland': [52.215933, 19.134422],
 'belgium': [50.6402809, 4.6667145],
 'norway': [60.5000209, 9.0999715],
 'france': [46.603354, 1.8883335],
 'italy': [42.6384261, 12.674297],
 'canada': [61.0666922, -107.9917071],
 'netherlands': [52.5001698, 5.7480821],
 'germany': [51.0834196, 10.4234469],
 'united states of america': [39.7837304, -100.4458825],
 'hungary': [47.1817585, 19.5060937],
 'israel': [31.5313113, 34.8667654],
 'switzerland': [46.7985624, 8.2319736],
 'india': [22.3511148, 78.6677428],
 'united arab emirates': [24.0002488, 53.9994829],
 'malaysia': [4.5693754, 102.2656823],
 'denmark': [55.670249, 10.3333283],
 'south africa': [-28.8166236, 24.991639],
 'rwanda': [-1.9646631, 30.0644358],
 'finland': [63.2467777, 25.9209164],
 'portugal': [40.0332629, -7.88962

In [ ]:
# Method 1 : apply row by row
def extract_address(row):
    address_split = row['Hotel_Address'].split()
    if address_split[-1] == 'Kingdom':
        country = ' '.join(address_split[-2:])
        row['City'] = ' '.join(address_split[-5:-4])
    else:
        country = address_split[-1]
        row['City'] = address_split[-2]
    row['Country'] = country

    reviewer_nationality = row['Reviewer_Nationality'].lower().strip()
    lat_lon = country_lat_lon_map.get(reviewer_nationality)
    if lat_lon is not None:
        lat, lon = lat_lon
    else:
        lat = 'null'
        lon = 'null'
    row['Reviewer_Lat'] = lat
    row['Reviewer_Lon'] = lon

    return row


HRF = HRF.progress_apply(extract_address, axis=1)
HRF.head()

In [7]:
# Method 2 : process columns
def map_reviewer_lat(nationality):
    nationality = nationality.lower().strip()
    lat_lon = country_lat_lon_map.get(nationality)
    if lat_lon is not None:
        lat, lon = lat_lon
        return lat
    else:
        return 'null'


def map_reviewer_lon(nationality):
    nationality = nationality.lower().strip()
    lat_lon = country_lat_lon_map.get(nationality)
    if lat_lon is not None:
        lat, lon = lat_lon
        return lat
    else:
        return 'null'


def map_hotel_country(address):
    address_split = address.split()
    if address_split[-1] == 'Kingdom':
        return ' '.join(address_split[-2:])
    else:
        return address_split[-1]


def map_hotel_city(address):
    address_split = address.split()
    if address_split[-1] == 'Kingdom':
        return ' '.join(address_split[-5:-4])
    else:
        return address_split[-2]


HRF = pd.read_csv('Hotel_Reviews.csv')

nationalities = HRF['Reviewer_Nationality'].copy().to_numpy()
lat_df = pd.DataFrame(
    list(map(map_reviewer_lat, nationalities)), columns=['Reviewer_Lat'])
lon_df = pd.DataFrame(
    list(map(map_reviewer_lon, nationalities)), columns=['Reviewer_Lon'])

hotel_address = HRF['Hotel_Address'].copy().to_numpy()
hotel_country_df = pd.DataFrame(
    list(map(map_hotel_country, hotel_address)), columns=['Country'])
hotel_city_df = pd.DataFrame(
    list(map(map_hotel_city, hotel_address)), columns=['City'])

HRF = pd.concat([HRF, lat_df, lon_df, hotel_country_df, hotel_city_df], axis=1)
HRF.head()

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,...,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng,Reviewer_Lat,Reviewer_Lon,Country,City
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,...,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968,64.6863,64.6863,Netherlands,Amsterdam
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,...,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968,52.8652,52.8652,Netherlands,Amsterdam
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,...,9,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,52.360576,4.915968,-24.7761,-24.7761,Netherlands,Amsterdam
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,Great location in nice surroundings the bar a...,...,1,3.8,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3 days,52.360576,4.915968,55.367,55.367,Netherlands,Amsterdam
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/24/2017,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,140,1403,Amazing location and building Romantic setting,...,3,6.7,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",10 days,52.360576,4.915968,-41.5001,-41.5001,Netherlands,Amsterdam


In [8]:
HRF['Country'].unique()

array(['Netherlands', 'United Kingdom', 'France', 'Spain', 'Italy',
       'Austria'], dtype=object)

In [9]:
HRF['City'].unique()

array(['Amsterdam', 'London', 'Paris', 'Barcelona', 'Milan', 'Vienna'],
      dtype=object)

In [10]:
HRF['Reviewer_Lat'].unique()

array([64.6863136, 52.865196, -24.7761086, 55.367000000000004,
       -41.5000831, 52.215933, 50.6402809, 60.5000209, 46.603354,
       42.6384261, 61.0666922, 52.5001698, 51.0834196, 39.7837304,
       47.1817585, 31.5313113, 46.7985624, 22.3511148, 24.0002488,
       4.5693754, 55.670249, -28.8166236, -1.9646631, 63.2467777,
       40.0332629, 49.8158683, 39.3262345, 35.8885993, 35.000074,
       38.9597594, 49.4871968, 36.10674695, -34.9964963, 'null',
       54.1936805, 59.6749712, 45.5643442, 25.6242618, 55.3500003,
       45.8133113, 8.559559, 47.2000338, -10.3333333, 23.9739374,
       64.9841821, 32.6475314, 47.1416307, -31.7613365, 31.1728205,
       21.0000287, 33.8750629, 25.3336984, 9.6000359, 45.9852129,
       49.8167003, 38.9953683, 14.8971921, 1.357107, 49.579520200000005,
       56.8406494, -23.2335499, 36.5748441, 26.2540493, 48.7411522,
       35.7724185, 29.2733964, 26.1551249, 49.1630548, 31.1667049,
       42.6073975, 34.9823018, 41.000028, 58.71971295, 44.1534121

In [11]:
HRF['Reviewer_Lon'].unique()

array([64.6863136, 52.865196, -24.7761086, 55.367000000000004,
       -41.5000831, 52.215933, 50.6402809, 60.5000209, 46.603354,
       42.6384261, 61.0666922, 52.5001698, 51.0834196, 39.7837304,
       47.1817585, 31.5313113, 46.7985624, 22.3511148, 24.0002488,
       4.5693754, 55.670249, -28.8166236, -1.9646631, 63.2467777,
       40.0332629, 49.8158683, 39.3262345, 35.8885993, 35.000074,
       38.9597594, 49.4871968, 36.10674695, -34.9964963, 'null',
       54.1936805, 59.6749712, 45.5643442, 25.6242618, 55.3500003,
       45.8133113, 8.559559, 47.2000338, -10.3333333, 23.9739374,
       64.9841821, 32.6475314, 47.1416307, -31.7613365, 31.1728205,
       21.0000287, 33.8750629, 25.3336984, 9.6000359, 45.9852129,
       49.8167003, 38.9953683, 14.8971921, 1.357107, 49.579520200000005,
       56.8406494, -23.2335499, 36.5748441, 26.2540493, 48.7411522,
       35.7724185, 29.2733964, 26.1551249, 49.1630548, 31.1667049,
       42.6073975, 34.9823018, 41.000028, 58.71971295, 44.1534121

In [12]:
HRF.to_csv('Hotel_Reviews_With_City.csv')

!head Hotel_Reviews_With_City.csv

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng,Reviewer_Lat,Reviewer_Lon,Country,City
0, s Gravesandestraat 55 Oost 1092 AA Amsterdam Netherlands,194,8/3/2017,7.7,Hotel Arena, Russia , I am so angry that i made this post available via all possible sites i use when planing my trips so no one will make the mistake of booking this place I made my booking via booking com We stayed for 6 nights in this hotel from 11 to 17 July Upon arrival we were placed in a small room on the 2nd floor of the hotel It turned out that this was not the room we booked I had specially reserved the 2 level duplex room so that we would have a big windows and high ceilings The room itself was ok if you don t mind the broken window that can not be clo

In [13]:
print(pd.read_csv('Hotel_Reviews.csv').shape[0])
print(pd.read_csv('Hotel_Reviews_With_City.csv').shape[0])

515738
515738
